# Day 14: Docking Data

## Part 1
Memory represented as 36-bit unsigned integers  
In other words 36 numbers as ones or zeros  
Input starts of with a 36-bit mask followed by values assigned to specific addresses in memory.  
What we have to do is to convert the value we try to store into binary, apply the mask, convert to a value, store in the memory location.  
Once this is done for all input data, sum the values which exist in memory.

In [1]:
import re

In [2]:
program = open("input/input-day-14.txt", "r").read().split("\n")[:-1]

In [4]:
pattern_1 = re.compile(r"(^mask) = (([0-1]|X)+)") # Pattern for finding if a line has a mask
pattern_2 = re.compile(r"^mem\[(\d+)\] = (\d+)") # Pattern for finding the memory address and the value stored at it

memory = {}

for line in program:
    match_mask = pattern_1.search(line)
    
    if(match_mask):
        mask = match_mask.group(2)
        continue
    
    match_mem = pattern_2.search(line)
    
    if(match_mem):
        memory_address = match_mem.group(1)
        value = int(match_mem.group(2))
        
        # Convert value to binary
        bin_value = bin(value)[2:] # Ignore the first two characters generated by bin()
        # Zero filled "36-bit" array
        bin_value_extended = [0 for x in range(36)] 
        # add our binary value to the appropriate location in the 36-bit array
        bin_value_extended[-len(bin_value):] = bin_value 

        # Apply mask
        for i, mask_val in enumerate([x for x in mask]):
            if mask_val == "X":
                continue
            bin_value_extended[i] = mask_val
        
        # Join our list to a single string
        new_bin = "".join(map(str,bin_value_extended))
        # Convert our binary value back to decimal
        new_val = int(new_bin, 2)
        
        # Store the new decimal value in its appropriate memory location
        memory[memory_address] = new_val


# Calculate the sum for our answer
sum_of_values = 0
for address in memory.keys():
    sum_of_values += memory[address]
    
print("Sum of all values in memory:", sum_of_values)

Sum of all values in memory: 17934269678453


## Part 2

    If the bitmask bit is 0, the corresponding memory address bit is unchanged.
    If the bitmask bit is 1, the corresponding memory address bit is overwritten with 1.
    If the bitmask bit is X, the corresponding memory address bit is floating.
    
Floating means it can be both 1 an 0, hence all combinations should be used to mask each memory address


If we for example have the following mask 1001X1XX02X. There are 4 values which can be both 1 and 0. This can be seen as a combinatorics problem. Where the question is   

Number of ways of taking ```k``` elements from a population of ```n``` elements.
In our case the order matters and we are putting the elements back in the pile so to speak. Hence the following formula can be used to get number of variations we should get.

$$N_{combinations} = n^k$$



To get all combinations using Python, the ```product()``` function from ```itertools``` can be used. 

In [30]:
import itertools
n = 2 # [0,1] our population
k = 3 # if we have 3 st X
combinations = list(itertools.product("01", repeat=k))
print("Combinations", combinations)
print("Size of list", len(combinations))
print("n^k =", n**k)

Combinations [('0', '0', '0'), ('0', '0', '1'), ('0', '1', '0'), ('0', '1', '1'), ('1', '0', '0'), ('1', '0', '1'), ('1', '1', '0'), ('1', '1', '1')]
Size of list 8
n^k = 8


In [51]:
pattern_1 = re.compile(r"(^mask) = (([0-1]|X)+)") # Pattern for finding if a line has a mask
pattern_2 = re.compile(r"^mem\[(\d+)\] = (\d+)") # Pattern for finding the memory address and the value stored at it

memory = {}

for line in program:
    
    match_mask = pattern_1.search(line)
    
    if(match_mask):
        mask = match_mask.group(2)
        
        nr_of_floating_addresses = mask.count("X")
        # Generate all combinations of floating addresses.
        # They are in order in which they should be applied to our masked addresses
        mask_combinations = list(itertools.product("01", repeat=nr_of_floating_addresses))
        
        print("Mask:", mask)
        print("Nr of floating addresses:", nr_of_floating_addresses)
        print("Nr of combinations:", len(mask_combinations))
        print("")
        
        continue
    
    match_mem = pattern_2.search(line)
    
    if(match_mem):
        memory_address = int(match_mem.group(1))
        value = int(match_mem.group(2))
        
        # Convert memory address to binary
        bin_mem = bin(memory_address)[2:] # Ignore the first two characters generated by bin()
        # Zero filled "36-bit" array
        bin_mem_extended = [0 for x in range(36)] 
        # add our binary value to the appropriate location in the 36-bit array
        bin_mem_extended[-len(bin_mem):] = bin_mem 
        
        # Apply the mask to our memory address. 
        for combination in mask_combinations:
            current_floating = 0 # Keep track of which floating address value we should use next time a floating mask value appears
            for i, mask_val in enumerate([x for x in mask]):
                if(mask_val == "0"):
                    continue
                if(mask_val == "X"):
                    bin_mem_extended[i] = combination[current_floating]
                    current_floating += 1
                if(mask_val == "1"):
                    bin_mem_extended[i] = 1
            
            # Join our list to a single string
            new_bin = "".join(map(str,bin_mem_extended))
            # Convert our binary value back to decimal
            new_mem = int(new_bin, 2)

            # Store the new decimal value in its appropriate memory location
            memory[new_mem] = value

        
# Calculate the sum for our answer
sum_of_values = 0
for address in memory.keys():
    sum_of_values += memory[address]
    
print("Sum of all values in memory:", sum_of_values)

Mask: 110100X1X01011X01X0X000111X00XX1010X
Nr of floating addresses: 9
Nr of combinations: 512

Mask: 11110X000010XX10X11X00X11010X0X00101
Nr of floating addresses: 8
Nr of combinations: 256

Mask: 11XXX0XX001X01101111X001000X01X10000
Nr of floating addresses: 9
Nr of combinations: 512

Mask: X11X0X00001010101XX101100X011X000XX1
Nr of floating addresses: 9
Nr of combinations: 512

Mask: 111100X00010101011X1XXXXX01X000001X1
Nr of floating addresses: 9
Nr of combinations: 512

Mask: XX11101X0011X01X1111X001X1X000101110
Nr of floating addresses: 8
Nr of combinations: 256

Mask: X11110X11X11X000110110010110011X0001
Nr of floating addresses: 5
Nr of combinations: 32

Mask: 111100100010X0101XX1110100000XXX0001
Nr of floating addresses: 6
Nr of combinations: 64

Mask: 1X1101010X1X111X110101X1010100010010
Nr of floating addresses: 5
Nr of combinations: 32

Mask: 10XXX01100101010X1X111010000101101X1
Nr of floating addresses: 6
Nr of combinations: 64

Mask: 111010X10010111X1111X0XXX00110110001
N

Mask: X11X000000XX11X011X1XX0110000X000001
Nr of floating addresses: 9
Nr of combinations: 512

Mask: X1X01001011X001001010110101100X10011
Nr of floating addresses: 4
Nr of combinations: 16

Mask: 100XX1111X0011100X01000X010000000101
Nr of floating addresses: 5
Nr of combinations: 32

Mask: 1110X001X010X0101X01110100000X10111X
Nr of floating addresses: 6
Nr of combinations: 64

Mask: 11110000X0100110XX11X0110100X0110001
Nr of floating addresses: 5
Nr of combinations: 32

Mask: 10X0000100100X101X1X1101110000X0X011
Nr of floating addresses: 6
Nr of combinations: 64

Mask: 1X01X001X0X01X1011110001100X0110X100
Nr of floating addresses: 7
Nr of combinations: 128

Mask: 1101101110XX1000X001101110000000X1X0
Nr of floating addresses: 5
Nr of combinations: 32

Mask: 111010010X1000100X01X1XX10XX0XX10000
Nr of floating addresses: 9
Nr of combinations: 512

Mask: X11XX111001001101101010101000X000101
Nr of floating addresses: 4
Nr of combinations: 16

Mask: 111X00100X0X1010110100001X011X1X00X0
Nr o